# Use SVM to predict the stock movement
## dataset
   [2009/1~2019/1](https://www.sharecast.com/index/SP_500/prices/download)


In [57]:
import pandas as pd

path = './download.csv'
pd_data = pd.read_csv(path)
pd_data.head()

import warnings
warnings.filterwarnings("ignore")

## create stock movement column
    和前一天比，increase:1, decrease:0

In [58]:
pd_data['movement'] = 0

for i in range(1, 2516):
    if(pd_data['Close Price'][i] > pd_data['Close Price'][i-1]):
        pd_data['movement'][i] = 1
    

pd_data.head()

,Date,Open Price,Close Price,High Price,Low Price,Volume,movement
0,02-Jan-2009,902.99,931.80,934.73,899.35,4048270080,0
1,05-Jan-2009,929.17,927.45,936.63,919.53,5413910016,0
2,06-Jan-2009,931.17,934.70,943.85,927.28,5392620032,1
3,07-Jan-2009,927.45,906.65,927.45,902.37,4704940032,0
4,08-Jan-2009,905.73,909.73,910.00,896.81,4991549952,1


## get train set and test set
    前2264筆資料當作train set，剛好是從2009~2017年，剩餘當作test set，2018年

In [59]:
#print(pd_data['Date'].size)
train = pd_data[:2264]
test = pd_data[2264:]

#train set x
xtr = train[['Open Price', 'Close Price', 'High Price', 'Low Price', 'Volume']]
#train set y
ytr = train[['movement']]
#test set x 
xte = test[['Open Price', 'Close Price', 'High Price', 'Low Price', 'Volume']]
#test set y
yte = test[['movement']]

print(xtr.shape)
print(ytr.shape)
print(xte.shape)
print(yte.shape)

(2264, 5)
(2264, 1)
(252, 5)
(252, 1)


## standardization

In [60]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(xtr)
xtr_std = scaler.transform(xtr)
xte_std = scaler.transform(xte)


xtr_std

array([[-1.55230001, -1.49441197, -1.50545617, -1.54093327,  0.81362581],
       [-1.49834121, -1.50337884, -1.50153623, -1.49936521,  1.82418053],
       [-1.49421907, -1.48843406, -1.48664047, -1.48340126,  1.80842623],
       ...,
       [ 2.1145693 ,  2.11464105,  2.10688987,  2.1247189 , -1.33171387],
       [ 2.12281357,  2.12478289,  2.11105738,  2.13250518, -1.34874724],
       [ 2.12909984,  2.09606828,  2.12025892,  2.11380162, -1.19609981]])

## 用SVM model預測
### 用預設的參數，發現準確率偏低
    default: penalty = 1, kernel = rbf

In [61]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm = SVC()
svm.fit(xtr_std, ytr)

acc_rate = accuracy_score(yte, svm.predict(xte_std))
print(acc_rate)
print(svm.predict(xte_std))

0.5277777777777778
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1]


### 換個kernel試試看
    kernel = linear

In [62]:
svm = SVC(kernel = 'linear')
svm.fit(xtr_std, ytr)

acc_rate = accuracy_score(yte, svm.predict(xte_std))
print(acc_rate)
print(svm.predict(xte_std))

0.7896825396825397
[1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 1 1 1 1 0 0 0 1
 1 0 0 0 1 1 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1
 0 0 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 0 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1
 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1
 0 1 1 1 1 1 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 1 1 0 1 1 1 1 0 0 0 0 1
 1 0 0 1 1 1 1 1 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1]
